In [1]:
import altair as alt
from altair.expr import datum, if_
import datetime
import json
import numpy as np
import pandas as pd
import psycopg2
pd.options.display.float_format = "{:,.2f}".format

Connect to the database.

In [2]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf["database"],
    user=conf["user"],
    host=conf["host"],
    password=conf["password"]
)
conn.autocommit = True

## Return the donors who gave at least $50,000 to conservative and GOP-aligned groups between Nov. 2 and Dec. 31, 2017, split by their giving before the tax bill's introduction on Nov. 2 and after its introduction. Do this for every off-year going back to the 2010 cycle.

In [3]:
def run_donors_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """+prefix+"""_donors_committees_"""+suffix+""" AS
                    SELECT match_id,
                           organizations,
                           contributors,
                           sum(committee_total) AS total,
                           cmte_id,
                           pacshort AS committee
                    FROM
                      (SELECT match_id,
                              organizations,
                              contributors,
                              sum(amount) AS committee_total,
                              cmteid AS cmte_id
                       FROM
                         (SELECT CASE
                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                     ELSE orgname
                                 END AS match_id,
                                 array_agg(DISTINCT orgname) AS organizations,
                                 array_agg(DISTINCT contrib) AS contributors
                          FROM crp_contributions
                          LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
                          AND crp_committees.cycle = '""" + reference_cycle + """'
                          WHERE primcode IN ('J1100',
                                             'J2200',
                                             'J2400',
                                             'Z1100',
                                             'Z4100',
                                             'Z4500',
                                             'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
                            AND TYPE NOT IN ('11J',
                                             '15C',
                                             '15E',
                                             '15I',
                                             '15J',
                                             '15T',
                                             '18J',
                                             '19J',
                                             '30J',
                                             '30F',
                                             '31J',
                                             '31F',
                                             '32J',
                                             '32F')
                          GROUP BY match_id
                          HAVING sum(amount) >= 50000) AS gop_donors
                       JOIN crp_contributions ON CASE
                                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                                     ELSE crp_contributions.orgname
                                                 END = gop_donors.match_id
                       WHERE date >= '""" + start + """'
                         AND date <= '""" + end + """'
                         AND CYCLE = '""" + cycle + """'
                         AND TYPE NOT IN ('11J',
                                          '15C',
                                          '15E',
                                          '15I',
                                          '15J',
                                          '15T',
                                          '18J',
                                          '19J',
                                          '30J',
                                          '30F',
                                          '31J',
                                          '31F',
                                          '32J',
                                          '32F')
                       GROUP BY match_id,
                                organizations,
                                contributors,
                                cmte_id) AS donors_committees
                    JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
                    WHERE primcode IN ('J1100',
                                       'J2200',
                                       'J2400',
                                       'Z1100',
                                       'Z4100',
                                       'Z4500',
                                       'Z5100')
                    GROUP BY match_id,
                             organizations,
                             contributors,
                             cmte_id,
                             committee;

                    SELECT *
                    FROM """+prefix+"""_donors_committees_"""+suffix+""";
                    """
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [4]:
post_bill_donors_committees_17 = run_donors_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_donors_committees_17 = run_donors_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

In [5]:
post_bill_donors_committees_15 = run_donors_query("post_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-11-02", "2015-12-31")
pre_bill_donors_committees_15 = run_donors_query("pre_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-01-01", "2015-11-01")

In [ ]:
post_bill_donors_committees_13 = run_donors_query("post_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-11-02", "2013-12-31")
pre_bill_donors_committees_13 = run_donors_query("pre_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-01-01", "2013-11-01")

In [ ]:
post_bill_donors_committees_11 = run_donors_query("post_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-11-02", "2011-12-31")
pre_bill_donors_committees_11 = run_donors_query("pre_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-01-01", "2011-11-01")

In [ ]:
post_bill_donors_committees_09 = run_donors_query("post_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-11-02", "2009-12-31")
pre_bill_donors_committees_09 = run_donors_query("pre_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-01-01", "2009-11-01")

In [12]:
post_bill_donors_17 = post_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 4 columns):
match_id         158 non-null object
organizations    158 non-null object
contributors     158 non-null object
total            158 non-null float64
dtypes: float64(1), object(3)
memory usage: 5.0+ KB


## How much did they give?

In [13]:
post_bill_donors_17["total"].sum()

35639563.0

## Had these megadonors given previously in 2017?

Return contributions by donor and committee.

In [14]:
pre_bill_donors_17 = pre_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 4 columns):
match_id         896 non-null object
organizations    896 non-null object
contributors     896 non-null object
total            896 non-null float64
dtypes: float64(1), object(3)
memory usage: 28.1+ KB


## How much did they give?

In [15]:
pre_bill_donors_17["total"].sum()

182827883.0

## How much did these donors give in the period leading up to the tax bill's introduction and in the period after its introduction?

In [ ]:
donors_17 = post_bill_donors_17.merge(pre_bill_donors_17, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_17.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_17.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_17["total_pre_bill"].fillna(0, inplace=True)
donors_17.info()

In [ ]:
donors_17["pct_post_bill"] = donors_17["total_post_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["pct_pre_bill"] = donors_17["total_pre_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["change"] = donors_17["total_post_bill"] - donors_17["total_pre_bill"]
donors_17["pct_change"] = (donors_17["total_post_bill"] - donors_17["total_pre_bill"]) / donors_17["total_pre_bill"].abs()
donors_17.sort_values("total_post_bill", ascending=False).head()

## How many of these donors gave all of their money between Nov. 2 and Dec. 31?

In [ ]:
donors_17[donors_17["pct_post_bill"] == 1].count()

## Had these megadonors given between Nov. 2 and Dec. 31, 2015?

Return contributions by donor and committee.

In [ ]:
post_bill_donors_committees_15 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_15 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2015-11-02'
     AND date <= '2015-12-31'
     AND CYCLE = '2016'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2016'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_15 TO redash_default;


SELECT *
FROM post_bill_donors_committees_15;""", con=conn)
post_bill_donors_committees_15.info()

Group by donor.

In [ ]:
post_bill_donors_15 = post_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_15.info()

## How much did they give?

In [ ]:
post_bill_donors_15["total"].sum()

## Had these megadonors given previously in 2015?

Return contributions by donor and committee.

In [ ]:
pre_bill_donors_committees_15 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_15 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2015-01-01'
     AND date < '2015-11-02'
     AND CYCLE = '2016'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2016'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_15 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_15;""", con=conn)
pre_bill_donors_committees_15.info()

Group by donor.

In [ ]:
pre_bill_donors_15 = pre_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_15.info()

## How much did they give?

In [ ]:
pre_bill_donors_15["total"].sum()

## How much did these donors give in each period?

In [ ]:
donors_15 = post_bill_donors_15.merge(pre_bill_donors_15, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_15["total_pre_bill"].fillna(0, inplace=True)
donors_15["total_post_bill"].fillna(0, inplace=True)
donors_15.info()

In [ ]:
donors_15["pct_post_bill"] = donors_15["total_post_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["pct_pre_bill"] = donors_15["total_pre_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["change"] = donors_15["total_post_bill"] - donors_15["total_pre_bill"]
donors_15["pct_change"] = (donors_15["total_post_bill"] - donors_15["total_pre_bill"]) / donors_15["total_pre_bill"].abs()
donors_15 = donors_15[["match_id", "organizations_post_bill", "contributors_post_bill",
                       "organizations_pre_bill", "contributors_pre_bill", "total_post_bill",
                       "total_pre_bill", "pct_post_bill", "pct_pre_bill", "change", "pct_change"]]
donors_15.head()

## How many of these donors gave all of their money between Nov. 2 and Dec. 31?

In [ ]:
donors_15[donors_15["pct_post_bill"] == 1].count()

## And did their giving patterns change between the cycles?

In [ ]:
donors_15_17 = donors_17.merge(donors_15, how="outer", on="match_id", suffixes=["_17", "_15"])
donors_15_17.drop(["change_17", "pct_change_17", "organizations_post_bill", "contributors_post_bill",
             "organizations_pre_bill", "contributors_pre_bill", "change_15", "pct_change_15"],
            axis=1, inplace=True)
donors_15_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"})
donors_15_17 = donors_15_17[["match_id", "contributors", "organizations", "total_pre_bill_15", "total_post_bill_15",
                 "pct_pre_bill_15", "pct_post_bill_15", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_15_17.info()

In [ ]:
donors_15_17["giving_change"] = np.where((donors_15_17["pct_post_bill_17"] > donors_15_17["pct_post_bill_15"]) | (donors_15_17["pct_pre_bill_15"].isnull()), "Increased",
                                  np.where(donors_15_17["pct_post_bill_17"] < donors_15_17["pct_post_bill_15"], "Decreased",
                                           np.where(donors_15_17["pct_post_bill_17"] == donors_15_17["pct_post_bill_15"], "Stayed the same",
                                                   "Other")))
donors_15_17.head(1)

## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in 2015?

In [ ]:
donors_15_17[donors_15_17["giving_change"] == "Increased"].count() / donors_15_17.count()

## And by how much did these individuals' giving increase?

In [ ]:
donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()

In [ ]:
donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()

In [ ]:
donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()

# What does this giving look like on a contribution-by-contribution level?

During the period after the tax bill was introduced?

In [ ]:
post_bill_contributions_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_17 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-11-02'
     AND date <= '2017-12-31'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_17 TO redash_default;


SELECT *
FROM post_bill_contributions_17;""", con=conn)
post_bill_contributions_17.info()

During the period before the tax bill was introduced?

In [ ]:
pre_bill_contributions_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_17 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-01-01'
     AND date < '2017-11-02'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_17 TO redash_default;


SELECT *
FROM pre_bill_contributions_17;""", con=conn)
pre_bill_contributions_17.info()

What about during the equivalent post-bill introduction period in 2015?

In [ ]:
post_bill_contributions_15 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_15 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2015-11-02'
     AND date <= '2015-12-31'
     AND CYCLE = '2016'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2016'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_15 TO redash_default;


SELECT *
FROM post_bill_contributions_15;""", con=conn)
post_bill_contributions_15.info()

In [ ]:
pre_bill_contributions_15 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_15 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2015-01-01'
     AND date < '2015-11-02'
     AND CYCLE = '2016'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2016'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_15 TO redash_default;


SELECT *
FROM pre_bill_contributions_15;""", con=conn)
pre_bill_contributions_15.info()

## Had these megadonors given between Nov. 2 and Dec. 31, 2013?

Return contributions by donor and committee.

In [ ]:
post_bill_donors_committees_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_13 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-11-02'
     AND date <= '2013-12-31'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_13 TO redash_default;


SELECT *
FROM post_bill_donors_committees_13;""", con=conn)
post_bill_donors_committees_13.info()

Group by donor.

In [ ]:
post_bill_donors_13 = post_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_13.info()

## How much did they give?

In [ ]:
post_bill_donors_13["total"].sum()

## Had these megadonors given previously in 2013?

Return contributions by donor and committee.

In [ ]:
pre_bill_donors_committees_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_13 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-01-01'
     AND date < '2013-11-02'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_13 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_13;""", con=conn)
pre_bill_donors_committees_13.info()

Group by donor.

In [ ]:
pre_bill_donors_13 = pre_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_13.info()

## How much did they give?

In [ ]:
pre_bill_donors_13["total"].sum()

## How much did these donors give in each period?

In [ ]:
donors_13 = post_bill_donors_13.merge(pre_bill_donors_13, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_13["total_pre_bill"].fillna(0, inplace=True)
donors_13["total_post_bill"].fillna(0, inplace=True)
donors_13.info()

In [ ]:
donors_13["pct_post_bill"] = donors_13["total_post_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["pct_pre_bill"] = donors_13["total_pre_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["change"] = donors_13["total_post_bill"] - donors_13["total_pre_bill"]
donors_13["pct_change"] = (donors_13["total_post_bill"] - donors_13["total_pre_bill"]) / donors_13["total_pre_bill"].abs()
donors_13 = donors_13[["match_id", "organizations_post_bill", "contributors_post_bill",
                       "organizations_pre_bill", "contributors_pre_bill", "total_post_bill",
                       "total_pre_bill", "pct_post_bill", "pct_pre_bill", "change", "pct_change"]]
donors_13.head()

## How many of these donors gave all of their money between Nov. 2 and Dec. 31?

In [ ]:
donors_13[donors_13["pct_post_bill"] == 1].count()

## And did their giving patterns change between the cycles?

In [ ]:
donors_13_17 = donors_17.merge(donors_13, how="outer", on="match_id", suffixes=["_17", "_13"])
donors_13_17.drop(["change_17", "pct_change_17", "organizations_post_bill", "contributors_post_bill",
             "organizations_pre_bill", "contributors_pre_bill", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors_13_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"})
donors_13_17 = donors_13_17[["match_id", "contributors", "organizations", "total_pre_bill_13", "total_post_bill_13",
                 "pct_pre_bill_13", "pct_post_bill_13", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_13_17.info()

In [ ]:
donors_13_17["giving_change"] = np.where((donors_13_17["pct_post_bill_17"] > donors_13_17["pct_post_bill_13"]) | (donors_13_17["pct_pre_bill_13"].isnull()), "Increased",
                                  np.where(donors_13_17["pct_post_bill_17"] < donors_13_17["pct_post_bill_13"], "Decreased",
                                           np.where(donors_13_17["pct_post_bill_17"] == donors_13_17["pct_post_bill_13"], "Stayed the same",
                                                   "Other")))
donors_13_17.head(1)

## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in 2013?

In [ ]:
donors_13_17[donors_13_17["giving_change"] == "Increased"].count() / donors_13_17.count()

## And by how much did these individuals' giving increase?

In [ ]:
donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()

In [ ]:
donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()

In [ ]:
donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()

# What does this giving look like on a contribution-by-contribution level during the equivalent post-bill introduction period in 2013?

In [ ]:
post_bill_contributions_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_13 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-11-02'
     AND date <= '2013-12-31'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_13 TO redash_default;


SELECT *
FROM post_bill_contributions_13;""", con=conn)
post_bill_contributions_13.info()

In [ ]:
pre_bill_contributions_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_13 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-01-01'
     AND date < '2013-11-02'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_13 TO redash_default;


SELECT *
FROM pre_bill_contributions_13;""", con=conn)
pre_bill_contributions_13.info()

## Had these megadonors given between Nov. 2 and Dec. 31, 2011?

Return contributions by donor and committee.

In [ ]:
post_bill_donors_committees_11 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_11 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2011-11-02'
     AND date <= '2011-12-31'
     AND CYCLE = '2012'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2012'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_11 TO redash_default;


SELECT *
FROM post_bill_donors_committees_11;""", con=conn)
post_bill_donors_committees_11.info()

Group by donor.

In [ ]:
post_bill_donors_11 = post_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_11.info()

## How much did they give?

In [ ]:
post_bill_donors_11["total"].sum()

## Had these megadonors given previously in 2011?

Return contributions by donor and committee.

In [ ]:
pre_bill_donors_committees_11 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_11 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2011-01-01'
     AND date < '2011-11-02'
     AND CYCLE = '2012'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2012'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_11 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_11;""", con=conn)
pre_bill_donors_committees_11.info()

Group by donor.

In [ ]:
pre_bill_donors_11 = pre_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_11.info()

## How much did they give?

In [ ]:
pre_bill_donors_11["total"].sum()

## How much did these donors give in each period?

In [ ]:
donors_11 = post_bill_donors_11.merge(pre_bill_donors_11, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_11["total_pre_bill"].fillna(0, inplace=True)
donors_11["total_post_bill"].fillna(0, inplace=True)
donors_11.info()

In [ ]:
donors_11["pct_post_bill"] = donors_11["total_post_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["pct_pre_bill"] = donors_11["total_pre_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["change"] = donors_11["total_post_bill"] - donors_11["total_pre_bill"]
donors_11["pct_change"] = (donors_11["total_post_bill"] - donors_11["total_pre_bill"]) / donors_11["total_pre_bill"].abs()
donors_11 = donors_11[["match_id", "organizations_post_bill", "contributors_post_bill",
                       "organizations_pre_bill", "contributors_pre_bill", "total_post_bill",
                       "total_pre_bill", "pct_post_bill", "pct_pre_bill", "change", "pct_change"]]
donors_11.head()

## How many of these donors gave all of their money between Nov. 2 and Dec. 31?

In [ ]:
donors_11[donors_11["pct_post_bill"] == 1].count()

## And did their giving patterns change between the cycles?

In [ ]:
donors_11_17 = donors_17.merge(donors_11, how="outer", on="match_id", suffixes=["_17", "_11"])
donors_11_17.drop(["change_17", "pct_change_17", "organizations_post_bill", "contributors_post_bill",
             "organizations_pre_bill", "contributors_pre_bill", "change_11", "pct_change_11"],
            axis=1, inplace=True)
donors_11_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"})
donors_11_17 = donors_11_17[["match_id", "contributors", "organizations", "total_pre_bill_11", "total_post_bill_11",
                 "pct_pre_bill_11", "pct_post_bill_11", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_11_17.info()

In [ ]:
donors_11_17["giving_change"] = np.where((donors_11_17["pct_post_bill_17"] > donors_11_17["pct_post_bill_11"]) | (donors_11_17["pct_pre_bill_11"].isnull()), "Increased",
                                  np.where(donors_11_17["pct_post_bill_17"] < donors_11_17["pct_post_bill_11"], "Decreased",
                                           np.where(donors_11_17["pct_post_bill_17"] == donors_11_17["pct_post_bill_11"], "Stayed the same",
                                                   "Other")))
donors_11_17.head(1)

## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in 2011?

In [ ]:
donors_11_17[donors_11_17["giving_change"] == "Increased"].count() / donors_11_17.count()

## And by how much did these individuals' giving increase?

In [ ]:
donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()

In [ ]:
donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()

In [ ]:
donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()

# What does this giving look like on a contribution-by-contribution level during the equivalent post-bill introduction period in 2011?

In [ ]:
post_bill_contributions_11 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_11 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2011-11-02'
     AND date <= '2011-12-31'
     AND CYCLE = '2012'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2012'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_11 TO redash_default;


SELECT *
FROM post_bill_contributions_11;""", con=conn)
post_bill_contributions_11.info()

In [ ]:
pre_bill_contributions_11 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_11 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2011-01-01'
     AND date < '2011-11-02'
     AND CYCLE = '2012'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2012'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_11 TO redash_default;


SELECT *
FROM pre_bill_contributions_11;""", con=conn)
pre_bill_contributions_11.info()

## Had these megadonors given between Nov. 2 and Dec. 31, 2009?

Return contributions by donor and committee.

In [ ]:
post_bill_donors_committees_09 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_09 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2009-11-02'
     AND date <= '2009-12-31'
     AND CYCLE = '2010'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2010'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_09 TO redash_default;


SELECT *
FROM post_bill_donors_committees_09;""", con=conn)
post_bill_donors_committees_09.info()

Group by donor.

In [ ]:
post_bill_donors_09 = post_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_09.info()

## How much did they give?

In [ ]:
post_bill_donors_09["total"].sum()

## Had these megadonors given previously in 2009?

Return contributions by donor and committee.

In [ ]:
pre_bill_donors_committees_09 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_09 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2009-01-01'
     AND date < '2009-11-02'
     AND CYCLE = '2010'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2010'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_09 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_09;""", con=conn)
pre_bill_donors_committees_09.info()

Group by donor.

In [ ]:
pre_bill_donors_09 = pre_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_09.info()

## How much did they give?

In [ ]:
pre_bill_donors_09["total"].sum()

## How much did these donors give in each period?

In [ ]:
donors_09 = post_bill_donors_09.merge(pre_bill_donors_09, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_09["total_pre_bill"].fillna(0, inplace=True)
donors_09["total_post_bill"].fillna(0, inplace=True)
donors_09.info()

In [ ]:
donors_09["pct_post_bill"] = donors_09["total_post_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["pct_pre_bill"] = donors_09["total_pre_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["change"] = donors_09["total_post_bill"] - donors_09["total_pre_bill"]
donors_09["pct_change"] = (donors_09["total_post_bill"] - donors_09["total_pre_bill"]) / donors_09["total_pre_bill"].abs()
donors_09 = donors_09[["match_id", "organizations_post_bill", "contributors_post_bill",
                       "organizations_pre_bill", "contributors_pre_bill", "total_post_bill",
                       "total_pre_bill", "pct_post_bill", "pct_pre_bill", "change", "pct_change"]]
donors_09.head()

## How many of these donors gave all of their money between Nov. 2 and Dec. 31?

In [ ]:
donors_09[donors_09["pct_post_bill"] == 1].count()

## And did their giving patterns change between the cycles?

In [ ]:
donors_09_17 = donors_17.merge(donors_09, how="outer", on="match_id", suffixes=["_17", "_09"])
donors_09_17.drop(["change_17", "pct_change_17", "organizations_post_bill", "contributors_post_bill",
             "organizations_pre_bill", "contributors_pre_bill", "change_09", "pct_change_09"],
            axis=1, inplace=True)
donors_09_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"})
donors_09_17 = donors_09_17[["match_id", "contributors", "organizations", "total_pre_bill_09", "total_post_bill_09",
                 "pct_pre_bill_09", "pct_post_bill_09", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_09_17.info()

In [ ]:
donors_09_17["giving_change"] = np.where((donors_09_17["pct_post_bill_17"] > donors_09_17["pct_post_bill_09"]) | (donors_09_17["pct_pre_bill_09"].isnull()), "Increased",
                                  np.where(donors_09_17["pct_post_bill_17"] < donors_09_17["pct_post_bill_09"], "Decreased",
                                           np.where(donors_09_17["pct_post_bill_17"] == donors_09_17["pct_post_bill_09"], "Stayed the same",
                                                   "Other")))
donors_09_17.head(1)

## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in 2009?

In [ ]:
donors_09_17[donors_09_17["giving_change"] == "Increased"].count() / donors_09_17.count()

## And by how much did these individuals' giving increase?

In [ ]:
donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()

In [ ]:
donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()

In [ ]:
donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()

# What does this giving look like on a contribution-by-contribution level during the equivalent post-bill introduction period in 2009?

In [ ]:
post_bill_contributions_09 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_09 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2009-11-02'
     AND date <= '2009-12-31'
     AND CYCLE = '2010'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2010'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_09 TO redash_default;


SELECT *
FROM post_bill_contributions_09;""", con=conn)
post_bill_contributions_09.info()

In [ ]:
pre_bill_contributions_09 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_09 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2009-01-01'
     AND date < '2009-11-02'
     AND CYCLE = '2010'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2010'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_09 TO redash_default;


SELECT *
FROM pre_bill_contributions_09;""", con=conn)
pre_bill_contributions_09.info()

## Export data to Excel.

In [ ]:
writer = pd.ExcelWriter("data/megadonors.xlsx")
donors_15_17.to_excel(writer, "donors_15_17", startcol=0, index=False)
donors_13_17.to_excel(writer, "donors_13_17", startcol=0, index=False)
donors_11_17.to_excel(writer, "donors_11_17", startcol=0, index=False)
donors_09_17.to_excel(writer, "donors_09_17", startcol=0, index=False)
post_bill_donors_committees_17.to_excel(writer, "post_bill_donors_committees_17", startcol=0, index=False)
pre_bill_donors_committees_17.to_excel(writer, "pre_bill_donors_committees_17", startcol=0, index=False)
post_bill_donors_committees_15.to_excel(writer, "post_bill_donors_committees_15", startcol=0, index=False)
pre_bill_donors_committees_15.to_excel(writer, "pre_bill_donors_committees_15", startcol=0, index=False)
post_bill_donors_committees_13.to_excel(writer, "post_bill_donors_committees_13", startcol=0, index=False)
pre_bill_donors_committees_13.to_excel(writer, "pre_bill_donors_committees_13", startcol=0, index=False)
post_bill_donors_committees_11.to_excel(writer, "post_bill_donors_committees_11", startcol=0, index=False)
pre_bill_donors_committees_11.to_excel(writer, "pre_bill_donors_committees_11", startcol=0, index=False)
post_bill_donors_committees_09.to_excel(writer, "post_bill_donors_committees_09", startcol=0, index=False)
pre_bill_donors_committees_09.to_excel(writer, "pre_bill_donors_committees_09", startcol=0, index=False)
post_bill_contributions_17.to_excel(writer, "post_bill_contributions_17", startcol=0, index=False)
pre_bill_contributions_17.to_excel(writer, "pre_bill_contributions_17", startcol=0, index=False)
post_bill_contributions_15.to_excel(writer, "post_bill_contributions_15", startcol=0, index=False)
pre_bill_contributions_15.to_excel(writer, "pre_bill_contributions_15", startcol=0, index=False)
post_bill_contributions_13.to_excel(writer, "post_bill_contributions_13", startcol=0, index=False)
pre_bill_contributions_13.to_excel(writer, "pre_bill_contributions_13", startcol=0, index=False)
post_bill_contributions_11.to_excel(writer, "post_bill_contributions_11", startcol=0, index=False)
pre_bill_contributions_11.to_excel(writer, "pre_bill_contributions_11", startcol=0, index=False)
post_bill_contributions_09.to_excel(writer, "post_bill_contributions_09", startcol=0, index=False)
pre_bill_contributions_09.to_excel(writer, "pre_bill_contributions_09", startcol=0, index=False)
writer.save()